### 9. 台灣杉三號 guppy-gpu 工作派送排程 (四)
    - 課程重點: 學習如何執行模組化派送工作到計算節點 (guppy-gpu)
    - 課程重點: 把執行的job 寫成FUNCTION, 利用環境參數製作工作內容
    - https://hackmd.io/@whYPD8MBSHWRZV6y-ymFwQ/SyW43dlOt

In [ ]:
# https://man.twcc.ai/@TWCC-III-manual/ryyo0tsuu
def SLURM(cmd):
    ## SLURM 內容, 請修改 ---> Email
    SLURM='''#!/work/c00cjz002/binary/bash5.0/bin/bash
#SBATCH -A GOV110079        # 計畫代號
#SBATCH -J GPU_T2        # 工作代號 (標籤, 可自行定義)
#SBATCH -p gp1d             # 工作區塊 
#SBATCH -c 4                # 使用的CPU核心數
#SBATCH --gres=gpu:1        # GPU數量, 記憶體容量x90GBxGPU core)
#SBATCH --mail-user=summerhill001@gmail.com    # 請修改為您的信向
#SBATCH --mail-type=BEGIN,END                  # 指定送出email時機 可為NONE, BEGIN, END, FAIL, REQUEUE, ALL
#SBATCH -o log/%j.logi      # 執行記錄檔案儲存於log目錄底下
'''
    myCmd = SLURM + cmd
    
    ## 儲存上述內容 SLURM.sh
    import time
    slurm_shell = 'slurm/'+time.strftime("%Y-%m-%d_%H-%M-%S")+'.sh'
    
    f = open(slurm_shell, "w")
    f.write(myCmd)
    f.close()    

    ## 執行SLURM
    #!sbatch SLURM.sh
    jobID=(subprocess.check_output('sbatch '+slurm_shell+' |awk \'{print $4}\'', shell=True,text=True))
    return jobID

## 建立目錄
import subprocess
!mkdir -p slurm
!mkdir -p log

In [2]:
def createCMD(inputFile, saveFolder):
    cmd='''  
echo "這是測試頁面05"
# 您的程式部分
## -> 切換目錄
%cd ~/class20211116

## -> 載入singularity, 利用singularity執行程式
ml singularity
singularity exec \
-B /work/c00cjz002/data/fasta5 \
$HOME/guppy-gpu_latest.sif \
guppy_basecaller \
--compress_fastq \
-i '''+inputFile+''' \
-s $HOME/'''+saveFolder+''' \
--flowcell FLO-MIN106 \
--kit SQK-LSK109 \
-x cuda:0

'''
    return cmd

In [3]:
## 指令集
import os
inputFile="/work/c00cjz002/data/fasta5/g1/fast5_small"
saveFolder="class20211116/output/"+os.path.splitext(os.path.basename(inputFile))[0]
cmd=createCMD(inputFile, saveFolder)
print(cmd)

  
echo "這是測試頁面05"
# 您的程式部分
## -> 切換目錄
%cd ~/class20211116

## -> 載入singularity, 利用singularity執行程式
ml singularity
singularity exec -B /work/c00cjz001/data $HOME/guppy-gpu_latest.sif guppy_basecaller --compress_fastq -i /work/c00cjz001/data/fast5_small -s $HOME/class20211116/output/fast5_small --flowcell FLO-MIN106 --kit SQK-LSK109 -x cuda:0




In [4]:
## 送出工作到計算節點電腦
jobID = SLURM(cmd)
print(jobID.strip())

321751


sbatch: INFO: It is recommended to specify `--nodes` and `--ntasks-per-node` together


In [5]:
## 觀看執行狀況
!squeue -u `whoami` | grep "$jobID\|JOBID" 

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            321751      gp1d   GPU_T2 c00cjz00 PD       0:00      1 (None)


In [ ]:
## 刪除JOB
!scancel $jobID

In [15]:
## 全部JOB刪除
!squeue -u `whoami` | grep -v JOBID  | awk '{print $1}' | xargs scancel  # 列出 PID 並砍掉 Process